# Who should read this?

Anybody who is interested with using algebraic design patterns in Python.
I have three kinds of people in mind as my target audience.

* Me :-) Lets be clear, similiar to many monad tutorials, this is mostly a record for my future self, to remind me that I once understood something about this. 
* People familliar with these concepts in haskell, wanting to see them in dynamically typed language like Python
* People interested in these algebraic patterns that know Pthon and don't have resources to learn i.e. haskell.

If you are somebody interested, but wants to see these in haskell rather than python, go read [Bartosz Milewski's Programming Cafe](https://bartoszmilewski.com/), or [haskellforall by Gabriel Gonzalez](http://www.haskellforall.com/).

As with the previous article, some of my examples will be attempts to translate some articles about haskell to python :)

# Introducing Functor

Category theory is usually concerned with relation between diferent categories, but for programming, we are usually concerned only with category of types and the unary functions that convert between them.

When programmer talks about relarion between two categories,
usually one will basically be his favourite language and second one
some fancy extension to that language.

Like said original Haskell and Haskell with I/O.

So what in practice is a fuctor?

In general, anything where you can define a sensible `map` operation.
For example:

* list
* stream
* generator
* tree

Sensible for list means, that it shouldn't matter whether you
strs.map(trim).map(uppercase)

or

strs.map(lambda x: uppercase(trim(x)))

# Do you even lift?

There is a recuring concept, where we use some bit of information about
our datatype to convert functions that know nothig about it, to work with it.

We call this *lifting*.

For example, because I know that lists have map function, I could lift i.e. `str` to work on lists of things.


In [2]:
def lift(fn):
    def liftedfn(x):
      return list(map(fn,x))
    return liftedfn 

In [3]:
lifted_str = lift(str)
lifted_str([1,2,3])

['1', '2', '3']

# Category theory!

This idea of translating objects from one domain to another is at the core of the mathematical subject of category theory. 

Category in mathemathics is defined by three things:
* collection of objects
* collection of possible transformations between these objects, called morphisms
* if we can transform object X to Y, and we can transform object Y to Z, there needs to be transformation from A to C in our transformation collection

You could draw a category using upper-case letters for objects, and arrows to signify possible transformations.

![Category](https://upload.wikimedia.org/wikipedia/commons/e/ef/Commutative_diagram_for_morphism.svg)

Functor is then mapping between categories. On the image you can see a category of red objects, with morphisms $f$, $g$ and three identity morphisms (i.e. arrows that transform and object to itself) and a functor $F$ that *lifts* the transformations from red category, to green, where there are only two objects.

![Functor](https://upload.wikimedia.org/wikibooks/en/3/36/Functor.png) 

# Why should we care about lawfulness?

If you look at the laws for both functor and applicative, they may seem arbitrary. Well, for functor, I have some intuition, because I had abstract algebra course at university and when I look at the laws my mins goes "This kida looks like definition of homomorphism, I remember those to be useful, I guess it makes sense then." I dont really remember the specifics, but because it is vaguely familliar, I am fine with it.

There is better reason to like lawful interfaces, though. Just by using the the laws of the interface, we can define new and usefull stuff.

Lets look at nesting of two different functors, where we would have T[U[A]]. It is easy to see, that for any a, T[U[a]] is a functor as well.

If map for T is tmap, and map for U is umap, we can define the tumap for the nesting of U in T like this:

```
def tumap(f,tu):
  return tmap(lambda x: umap(f,x), tu)
```

We could write this as:

$$ \begin{align*}  \text{map}_{t_u}(f, t_u) &= {map}_{t} ( \lambda x: map_u(f,x), t_u) \end{align*} $$

And because we know the functor laws we can try to prove that they hold for this implementation as well.

* for any `tu`, `tumap(id, tu) == tu`

$$ \begin{align*} \text{map}_{t_u}(id, t_u) &= {map}_{t} ( \lambda x: map_u(id,x), t_u) \\
&= {map}_{t} ( \lambda x: x, t_u)  \\
&= {map}_{t} ( id, t_u) \\
&=  t_u \\
\end{align*} $$

* for any t and any two composable functions f and g, tumap(f,tumap(g,tu)) == map(compose(f,g),tu)

$$ \begin{align*}  \text{map}_{t_u}(f, {map}_{tu}(g,t_u)) &= \text{map}_{t_u}(f, {map}_{tu}(g,t_u)) \\
&= {map}_{t} ( \lambda x: map_u(f,x), {map}_{t}( \lambda x: map_u(g,x),t_u)) \\
&= {map}_{t}((\lambda x: map_u(f,x)) \circ (\lambda x: map_u(g,x)), t_u) \\ 
&= {map}_{t}(\lambda x: map_u(f,map_u(g,x)), t_u) \\
&= {map}_{t} ( \lambda x: map_u(compose(f,g),x), t_u) \\
&= \text{map}_{t_u}(compose(f,g), t_u) 
\end{align*} $$

Because we have laws for the interface, we do these kinds of self-contained proofs.

Similarily, you could prove, that if you have two applicatives nested, you still can operate the whole nested structure as applicative.


Another interesting thing that you could prove, that you can define `map` in terms of `apply` and `pure`:

```
def map(f, l):
  return apply(pure(f),l)
```
